<a href="https://colab.research.google.com/github/ashishkej-tsai/eva8_session1/blob/main/EVA8_Session_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function         # to bring the print function from Python 3 into Python 2
import torch                                  # import pytorch python package to create the network
import torch.nn as nn                         # torch.nn is a module in the PyTorch library which provides classes and functions for defining and initializing neural networks, as well as tools for training and evaluating models
import torch.nn.functional as F               # This module contains all the functions in the torch.nn library (whereas other parts of the library contain classes)
import torch.optim as optim                   # torch.optim is a package in the PyTorch library that contains optimization algorithms for training neural networks. It is a popular choice for performing optimization over neural network parameters and is often used in combination with a learning rate scheduler to adjust the learning rate during training. Some of the commonly used optimization algorithms in torch.optim include stochastic gradient descent (SGD), Adam, and Adagrad.
from torchvision import datasets, transforms  # torchvision.datasets is a package in the PyTorch library that contains popular datasets for computer vision tasks. torchvision.transforms, on the other hand, is a package that contains functions for transforming images, such as resizing and normalizing. Together, torchvision.datasets and torchvision.transforms can be used to easily load and preprocess data for training and evaluating deep learning models for computer vision tasks.

In [ ]:
# Building Main CNN Neural Network Model using torch nn library
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28x28x1 OUtput-28x28x32 RF 3x3   
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  #input - 28x28x32 OUtput-28x28x64 RF 5x5  
        self.pool1 = nn.MaxPool2d(2, 2)               #input - 28x28x64 OUtput-14x14x64 RF 10x10 
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14x14x64 OUtput-14x14x128 RF 12x12 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input - 14x14x128 OUtput-14x14x256 RF 14x14 
        self.pool2 = nn.MaxPool2d(2, 2)               #input - 14x14x256 OUtput-7x7x256 RF 28x28 
        self.conv5 = nn.Conv2d(256, 512, 3)           #input - 7x7x256 OUtput-5x5x512 RF 30x30 
        self.conv6 = nn.Conv2d(512, 1024, 3)          #input - 5x5x512 OUtput-3x3x1024 RF 32x32 
        self.conv7 = nn.Conv2d(1024, 10, 3)           #input - 3x3x1024 OUtput-1x1x10 RF 34x34 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # Building 1st block with relu activations
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # Building 2nd block with relu activations
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             # Building 3rd block with relu activations 
        x = F.relu(self.conv7(x))                                 # Output Layer giving with 10 output neurons
        x = x.view(-1, 10)                                        # Reshaping 1x1x10 to 10
        return F.log_softmax(x)                                   # Applies a softmax followed by a logarithm.While mathematically equivalent to log(softmax(x)), doing these two operations separately is slower and numerically unstable. This function uses an alternative formulation to compute the output and gradient correctly.

In [ ]:
!pip install torchsummary                             # torchsummary is a third-party package that provides a simple and easy-to-use interface for summarizing the details of a PyTorch model. It can be used to quickly and easily get a summary of a model's layers and parameters, including the number of trainable parameters, the input and output shapes for each layer, and the total number of parameters in the model. This can be useful for quickly understanding the structure of a model and for debugging.
from torchsummary import summary                      # Torch-summary provides information complementary to what is provided by print(your_model) in PyTorch, similar to Tensorflow's model.summary() API to view the visualization of the model, which is helpful while debugging your network. In this project, we implement a similar functionality in PyTorch and create a clean, simple interface to use in your projects.
use_cuda = torch.cuda.is_available()                  # API to check whether NVIDIA GPU cuda cores are available
device = torch.device("cuda" if use_cuda else "cpu")  # If NVIDIA GPU are not available then use the CPU
model = Net().to(device)                              # Creating the Torch Main model and sending it to either CPU or GPU (device)
summary(model, input_size=(1, 28, 28))                # Printing the model details using input size of 28x28x1

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)